<a href="https://colab.research.google.com/github/suryogumilar/Tensorflow_timeseries/blob/main/C4_W4_Lab_1_LSTM_andConvoNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using a Convolutions with multi-layer LSTM for forecasting



In [36]:
import tensorflow as tf
import numpy as np
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta
# for timezone()
import pytz
import math
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import plotly.graph_objects as go
from IPython.display import clear_output
import random
import os
from sklearn.preprocessing import MinMaxScaler

## Function

### Plot functions


In [37]:
def plot_series(time, series, format="-", start=0, end=None):
    """
    Visualizes time series data

    Args:
      time (array of int) - contains the time steps
      series (array of int) - contains the measurements for each time step
      format - line style when plotting the graph
      label - tag for the line
      start - first time step to plot
      end - last time step to plot
    """

    # Setup dimensions of the graph figure
    plt.figure(figsize=(10, 6))
    
    if type(series) is tuple:

      for series_num in series:
        # Plot the time series data
        plt.plot(time[start:end], series_num[start:end], format)

    else:
      # Plot the time series data
      plt.plot(time[start:end], series[start:end], format)

    # Label the x-axis
    plt.xlabel("Time")

    # Label the y-axis
    plt.ylabel("Value")

    # Overlay a grid on the graph
    plt.grid(True)

    # Draw the graph on screen
    plt.show()

In [38]:
def plot_series_plotly(time, series, series_name=None, 
                       figure_title='', showlegend=False, 
                       start=0, end=None,
                       xaxis_title="Time",
                       yaxis_title="Value"):
    """
    Visualizes time series data but using plotly for interactive graph

    Args:
      time (array of int) - contains the time steps
      series (array of int) - contains the measurements for each time step
      series_name (array of string) - contains correlative name of each series
      format - line style when plotting the graph
      label - tag for the line
      start - first time step to plot
      end - last time step to plot
    """
    fig = go.Figure()
    # Setup dimensions of the graph figure
    
    if type(series) is tuple:
      ii = 0
      for series_num in series:
        # Plot the time series data
        fig.add_trace(go.Scatter(x=time[start:end],
                                 y=series_num[start:end], mode='lines',
                                 name=series_name[ii]))  
        ii = ii+1
    else:
      # Plot the time series data
      fig.add_trace(go.Scatter(x=time[start:end],
                                 y=series[start:end], mode='lines'))

    fig.update_layout(title=figure_title, xaxis_title=xaxis_title, 
                      yaxis_title=yaxis_title,
                      autosize=False,
                      width=600,
                      height=600,
                      margin=dict(
                        l=50,
                        r=50,
                        b=100,
                        t=100,
                        pad=4
                        ), paper_bgcolor="LightSteelBlue"
                        , showlegend=showlegend
                      )
    fig.show()

In [39]:
def plot_candlesticks(df, figure_title='', showlegend=False):
  fig = go.Figure(data= [go.Candlestick(x=df['Date'],
                             open=df['Open'],
                             high=df['High'],
                             low=df['Low'],
                             close=df['Close']
                             )])
  fig.update_layout(title=figure_title, xaxis_title="Time", yaxis_title="Value",
                    autosize=False,
                    width=600,
                    height=600,
                    margin=dict(
                        l=50,
                        r=50,
                        b=100,
                        t=100,
                        pad=4
                    ),
                    paper_bgcolor="LightSteelBlue", showlegend=showlegend
                  )
  fig.show()

In [40]:
def plot_loss_inlog(history, epoch_value, lrs_value=1e-8, 
                    x_boundary1=1e-8, x_boundary2=1e-3,
                    y_boundary1=0, y_boundary2=50):
  """
  plot loss value after training in logaritmic scale

  Parameters:
    lrs_value: learning rate value that passed to LearningRateScheduler function
  """
  # Define the learning rate array
  lrs = lrs_value * (10 ** (np.arange(epoch_value) / 20))

  # Set the figure size
  plt.figure(figsize=(10, 6))

  # Set the grid
  plt.grid(True)

  # Plot the loss in log scale
  plt.semilogx(lrs, history.history["loss"])

  # Increase the tickmarks size
  plt.tick_params('both', length=10, width=1, which='both')

  # Set the plot boundaries
  plt.axis([x_boundary1, x_boundary2, 
            y_boundary1, y_boundary2])

In [41]:
def plot_loss_inlog_plotly(history, epoch_value, lrs_value=1e-8,
                           figure_title='Loss value', showlegend=False):
  """
  plot loss value after training in logaritmic scale

  Parameters:
    lrs_value: learning rate value that passed to LearningRateScheduler function
  """
  # Define the learning rate array
  lrs = lrs_value * (10 ** (np.arange(epoch_value) / 20))
  fig = go.Figure()
  fig.add_trace(go.Scatter(x=lrs,
                           y=history.history['loss'], mode='lines'))
  fig.update_xaxes(title_text="learning rate", type="log")
  
  fig.update_layout(title=figure_title, xaxis_title="Time", yaxis_title="Value",
                      autosize=False,
                      width=600,
                      height=600,
                      margin=dict(
                        l=50,
                        r=50,
                        b=100,
                        t=100,
                        pad=4
                        ), paper_bgcolor="LightSteelBlue"
                        , showlegend=showlegend
                      )
  fig.show()

In [42]:
def plot_prediction_graph(model, df, training_ds_rows,
                          window_size, 
                          normalizer_univar, denormalizer_univar,
                          variable_names='Close'):
  # Initialize a list
  forecast = []
  dataset_to_forecast = df[variable_names].iloc[training_ds_rows-window_size:]
  dateset_to_forecast_normalized = normalizer_univar(dataset_to_forecast)
  for time in range(dateset_to_forecast_normalized.shape[0] - window_size):
    the_prediction = model.predict(
        np.expand_dims(dateset_to_forecast_normalized[time:time + window_size], 
                      axis=0), 
        verbose=0)
    the_prediction_denorm = denormalizer_univar(the_prediction)
    forecast.append(the_prediction_denorm)
    
  # Convert to a numpy array and drop single dimensional axes
  results = np.array(forecast).squeeze()

  # Overlay the results with the validation set
  test_set = tf.convert_to_tensor(df[training_ds_rows:][variable_names])
  plot_series(df_test['Date'], (test_set, results) )
  return (test_set, results)

In [43]:
def plot_prediction_graph_plotly(model, df, training_ds_rows, 
                          window_size, normalizer_univar, denormalizer_univar,
                          scaler=None,
                          variable_names='Close',
                          series_name=['test dataset', 'predicted value']):
  # Initialize a list
  forecast = []
  dataset_to_forecast = df[variable_names].iloc[training_ds_rows-window_size:]
  if(normalizer_univar is not None):
    dateset_to_forecast_normalized = normalizer_univar(dataset_to_forecast)
  else:
    dateset_to_forecast_normalized = scaler.transform(dataset_to_forecast.values)
    print('dateset_to_forecast shape:', dateset_to_forecast_normalized.shape)
    print('expanded dateset_to_forecast shape:',
          np.expand_dims(dateset_to_forecast_normalized[0:window_size], 
                      axis=0).shape)
  for time in range(dateset_to_forecast_normalized.shape[0] - window_size):
    the_prediction = model.predict(
        np.expand_dims(dateset_to_forecast_normalized[time:time + window_size], 
                      axis=0), 
        verbose=0)
    if(denormalizer_univar is not None):
      the_prediction_denorm = denormalizer_univar(the_prediction)
    else:
      the_prediction_denorm = scaler.inverse_transform(the_prediction)
    forecast.append(the_prediction_denorm)
    
  # Convert to a numpy array and drop single dimensional axes
  print('forecast shape', np.array(forecast).shape)
  results = np.array(forecast).squeeze()
  print('forecast squeezed shape', results.shape)
  # Overlay the results with the validation set
  test_set = tf.convert_to_tensor(df[training_ds_rows:][variable_names])
  print('test_set.size', test_set.get_shape())
  print('results.ndim', results.ndim)
  
  # if(test_set.size==1):
  #   test_set = test_set.reshape(-1, 1)
  if(results.ndim==1):
    results = results.reshape(-1, len(variable_names))
    
  plot_series_plotly(df_test['Date'], (test_set[:,-1], results[:,-1]), 
                     series_name=series_name )
  return (test_set, results)
     

### function data retrieval 

In [44]:
def getStockData(history_span:int, the_ticker:str):
  """
  Getting stock data from Yahoo Finance API

  Args:
    history_span (int) how much backdate data to be collected
    the_ticker (string) ticker name on yahoo finance API
  Returns:
    Pandas dataframe (pd.DataFrame) containing stock data    
  """
  THE_URL = ('https://query1.finance.yahoo.com/v7/finance/'+
           'download/{ticker}?period1={period1}&period2={period2}&interval=1d&events=history&includeAdjustedClose=true')
  tdy = datetime.datetime.now(tz=pytz.timezone('Asia/Jakarta'))

  p2 = math.ceil(tdy.timestamp())
  p1 = math.floor((tdy - relativedelta(years=history_span)).timestamp())
  yf_url = THE_URL.format(ticker=the_ticker,period1=p1, period2=p2)
  df = pd.read_csv(yf_url)
  df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
  return df
     

### Fucntion to Make windowed data for time series forecasting

In [45]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    """Generates dataset windows

    Args:
      series (array of float) - contains the values of the time series
      window_size (int) - the number of time steps to include in the feature
      batch_size (int) - the batch size
      shuffle_buffer(int) - buffer size to use for the shuffle method

    Returns:
      dataset (TF Dataset) - TF Dataset containing time windows
    """
  
    # Generate a TF Dataset from the series values
    dataset = tf.data.Dataset.from_tensor_slices(series)
    
    # Window the data but only take those with the specified size
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    
    # Flatten the windows by putting its elements in a single batch
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))

    # Create tuples with features and labels 
    dataset = dataset.map(lambda window: (window[:-1], window[-1]))

    # Shuffle the windows
    dataset = dataset.shuffle(shuffle_buffer)
    
    # Create batches of windows
    dataset = dataset.batch(batch_size).prefetch(1)
    
    return dataset

### Tensorflow functions

for set random seed

In [46]:
def set_seed(seed: int = 42) -> None:
  random.seed(seed)
  np.random.seed(seed)
  tf.random.set_seed(seed)
  tf.experimental.numpy.random.seed(seed)
  #tf.keras.utils.set_random_seed(seed)
  #tf.config.experimental.enable_op_determinism()
  try:
    tf.set_random_seed(seed)
  except AttributeError as ae:
    print('INFO: tf.set_random_seed is deprecated in tf version ', tf.__version__, ' ',ae )
  
  # When running on the CuDNN backend, two further options must be set
  os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
  os.environ['TF_DETERMINISTIC_OPS'] = '1'
  # Set a fixed value for the hash seed
  os.environ["PYTHONHASHSEED"] = str(seed)
  print(f"Random seed set as {seed}")
     

## get data

In [47]:
# stock_name = str(input("Stock tick:"))
# hist_data = int(input("historical data (year):"))

stock_name = 'TLKM.JK'
hist_data = 3

In [48]:
df = getStockData(hist_data, stock_name)

## Split the Dataset

In [49]:
split_ratio = 0.8
rows_of_dataframe = df.shape[0]

training_ds_rows = round(rows_of_dataframe * split_ratio)
test_ds_rows = round(rows_of_dataframe * (1- split_ratio))

df_training = df[:training_ds_rows]
df_test = df[training_ds_rows:]

## Normalize

In [50]:
# convert to tensor
training_set = tf.convert_to_tensor(df_training['Close'])
print(training_set.shape)

(587,)


In [51]:
plot_series_plotly(df_training['Date'], training_set,
                   series_name='Close', figure_title='Stocks')

In [52]:
np.expand_dims(training_set, axis=1).shape

(587, 1)

In [53]:
minmax_scaler = MinMaxScaler(feature_range=(-1, 1))

training_set_normalized = minmax_scaler.fit_transform(np.expand_dims(training_set, axis=1))

In [54]:
plot_series_plotly(df_training['Date'], training_set_normalized[:,0],
                   series_name='Close', figure_title='Data Normalized')

## Prepare features and labels (windowed data)

In [58]:
window_size = 20
batch_size = 32
shuffle_buffer_size = 1000
     
## CONSTANT
MU = 0.000001
NANO = 1e-9

In [59]:
# Generate the dataset windows
windowed_training_ds = windowed_dataset(training_set_normalized, window_size, 
                           batch_size, shuffle_buffer_size)

In [60]:
# Print properties of a single batch
for windows in windowed_training_ds.take(1):
  print(f'data type: {type(windows)}')
  print(f'number of elements in the tuple: {len(windows)}')
  print(f'shape of first element: {windows[0].shape}')
  print(f'shape of second element: {windows[1].shape}')
  print(f'shape of first element expanded: {tf.expand_dims(windows[0], axis=-1).shape}')

data type: <class 'tuple'>
number of elements in the tuple: 2
shape of first element: (32, 20, 1)
shape of second element: (32, 1)
shape of first element expanded: (32, 20, 1, 1)


Since we use `MinMaxScaler` the shape of the input element already has the shape of `(batch_size, window_size, feature_size)` so we won't need flatten layer in fron of lstm layer

## Build and compile the model

Here is the model architecture you will be using. It is very similar to the last RNN you built but with the [Conv1D](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv1D) layer at the input. One important [argument](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv1D#args) here is the `padding`. For time series data, it is good practice to not let computations for a particular time step to be affected by values into the future. Here is one way of looking at it:

* Let's say you have a small time series window with these values: `[1, 2, 3, 4, 5]`. This means the value `1` is at `t=0`, `2` is at `t=1`, etc.
* If you have a 1D kernel of size `3`, then the first convolution will be for the values at `[1, 2, 3]` which are values for `t=0` to `t=2`.
* When you pass this to the first timestep of the `LSTM` after the convolution, it means that the value at `t=0` of the LSTM depends on `t=1` and `t=2` which are values into the future.
* For time series data, you want computations to only rely on current and previous time steps.
* One way to do that is to pad the array depending on the kernel size and stride. For a kernel size of 3 and stride of 1, the window can be padded as such: `[0, 0, 1, 2, 3, 4, 5]`. `1` is still at `t=0` and two zeroes are prepended to simulate values in the past.
* This way, the first stride will be at `[0, 0, 1]` and this does not contain any future values when it is passed on to subsequent layers.

The `Conv1D` layer does this kind of padding by setting `padding=causal` and you'll see that below.

In [63]:
class TS_LSTM_Model(tf.keras.Model):
  def __init__(self, window_size,feature_size=1,
               **kwargs):
    super(TS_LSTM_Model, self).__init__(**kwargs)

    model_tune = tf.keras.models.Sequential()
    model_tune.add(
      tf.keras.layers.Conv1D(filters=64, kernel_size=3,
                             strides=1, padding="causal",
                             activation="relu",
                             input_shape=[window_size, feature_size])
    )
    model_tune.add(
        tf.keras.layers.LSTM(64, return_sequences=True))
    model_tune.add(tf.keras.layers.LSTM(64))
    model_tune.add(tf.keras.layers.Dense(feature_size))
    self.seq_1 = model_tune

  @tf.function(reduce_retracing=True)   
  def call(self, x):
    x = self.seq_1(x)
    return x

## create model

In [64]:
learning_rate_value_all = 1e-6
epoch_value = 100

Since we are using `MinmaxScaler` we already reshaped the input dimension into `(Batch_size, window_size, feature_size)` so we do not need to add lambda expand dimension layer, so we set `expand_dims_layer=False` to the parameter of the model

and input shape for `build()` would be `input_shape = (1, window_size, feature_size)`

In [65]:
training_set_normalized.shape

(587, 1)

Before tuning, you can use the [`get_weights()`]((https://www.tensorflow.org/api_docs/python/tf/keras/layers/Layer#get_weights)) method so you can reset it later.

In [66]:
# Reset states generated by Keras
tf.keras.backend.clear_session()

model_tune2 = TS_LSTM_Model(window_size=window_size,
                              feature_size=training_set_normalized.shape[1])

In [68]:
# Get initial weights
init_weights = model_tune2.get_weights()

In [69]:

# set built parameter

print(training_set.shape)
model_tune2.build(input_shape = (1, window_size, training_set_normalized.shape[1]))
model_tune2.summary()
# Set the training parameters
model_tune2.compile(loss=tf.keras.losses.Huber(), 
                    optimizer=tf.keras.optimizers.SGD(
                        learning_rate=learning_rate_value_all,
                        momentum=0.9))

(587,)
Model: "ts_lstm__model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 1)                 66369     
                                                                 
Total params: 66,369
Trainable params: 66,369
Non-trainable params: 0
_________________________________________________________________


### Tune the Learning Rate

As usual, you will pick a learning rate by running the tuning code below.

In [70]:
# Set the learning rate scheduler
lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: learning_rate_value_all * 10**(epoch / 20))

# Train the model
history_tune2 = model_tune2.fit(windowed_training_ds, epochs=epoch_value, 
                                callbacks=[lr_schedule],
                                verbose=0)

In [71]:
plot_loss_inlog_plotly(history_tune2, epoch_value, lrs_value=learning_rate_value_all)

## Train the Model

proceed to train the model with your chosen learning rate. Choose the lowest loss value where the graph still smooth 

Tip: When experimenting and you find yourself running different iterations of a model, you may want to use the `clear_session()` method to declutter memory used by Keras. This is added in the first line below.


To reset the weights, you can simply call the [`set_weights()`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Layer#set_weights) and pass in the saved weights from earlier.

In [73]:
# Reset states generated by Keras
tf.keras.backend.clear_session()

model_tune2 = TS_LSTM_Model(window_size=window_size,
                            feature_size=training_set_normalized.shape[1])
#model_tune2.load_weights('./checkpoints/my_checkpoint')

model_tune2.set_weights(init_weights)

# set built parameter
print(training_set_normalized.shape)
model_tune2.build(input_shape = (1, window_size, training_set_normalized.shape[1]))
#model_tune2.summary()


# Set the training parameters
# usinbg mae as metrics
model_tune2.compile(loss=tf.keras.losses.Huber(), 
                    optimizer=tf.keras.optimizers.SGD(
                        learning_rate=0.003548,
                        momentum=0.9), metrics=["mae"])



(587, 1)


In [74]:

# Train the model
history_build1 = model_tune2.fit(windowed_training_ds, epochs=epoch_value, 
                                verbose=0)

In [75]:
history_build1.history.keys()

dict_keys(['loss', 'mae'])

In [77]:
plot_series_plotly(np.arange(0, epoch_value), 
                   (history_build1.history['loss'], history_build1.history['mae']),
                   figure_title='Mae and loss value',
                   series_name=['Loss Value', 'MAE Value'],
                   xaxis_title='epoch', yaxis_title='Loss')

In [78]:
test_set_g2, results2 = plot_prediction_graph_plotly(
    model_tune2, df, training_ds_rows, 
    window_size, None, None,
    scaler=minmax_scaler,
    variable_names=['Close'],
    series_name=['test set (close)',
                 'pred set (close)']
    )

dateset_to_forecast shape: (167, 1)
expanded dateset_to_forecast shape: (1, 20, 1)
forecast shape (147, 1, 1)
forecast squeezed shape (147,)
test_set.size (147, 1)
results.ndim 1


In [79]:
print(history_build1.history['loss'][-1])
print(tf.keras.metrics.mean_squared_error(test_set_g2[:,-1], results2[:,-1]).numpy())
print(tf.keras.metrics.mean_absolute_error(test_set_g2[:,-1], results2[:,-1]).numpy())

0.00625664321705699
16564.664
97.59266


## tomorrow pred

In [80]:
test_set = tf.convert_to_tensor(df_test[['Close']])

In [81]:
with np.printoptions(precision=0, suppress=True):
    print(test_set[-1].numpy())

[3880.]


In [82]:
test_set_normalized = minmax_scaler.transform(test_set)
test_set_normalized[-1]

array([0.19457014])

In [83]:
test_set_normalized.shape

(147, 1)

In [84]:
np.expand_dims(test_set_normalized[-window_size:], axis=0).shape

(1, 20, 1)

In [85]:
tomorro_pred = minmax_scaler.inverse_transform(model_tune2.predict(
    np.expand_dims(test_set_normalized[-window_size:], axis=0)))

1/1 [==============================] - 0s 23ms/step


In [86]:
with np.printoptions(precision=0, suppress=True):
    print(tomorro_pred)

#for elem in tomorro_pred[0]:
#  print('{:.1f}'.format(elem))

[[3966.]]


## save model

there are two method to save a model:

by calling `save()` function
by calling `save_weights()` function

In [87]:

model_tune2.save('./model/lstm01')

In [88]:
model_tune2.save_weights('./weights/lstm01')

## load model

by calling `tf.keras.models.load_model`

In [89]:
model_loaded0 = tf.keras.models.load_model('./model/lstm01')

In [90]:
tomorrow_pred = minmax_scaler.inverse_transform(model_loaded0.predict(
    np.expand_dims(test_set_normalized[-window_size:], axis=0)))

1/1 [==============================] - 0s 307ms/step


In [91]:
for elem in tomorro_pred[0]:
  print('{:.0f}'.format(elem))

3966


### by loading weights

In [93]:
model_loaded = TS_LSTM_Model(window_size=window_size, 
                             feature_size=training_set_normalized.shape[1]
                             )

In [94]:

model_loaded.load_weights('./weights/lstm01')

In [95]:
tomorrow_pred = minmax_scaler.inverse_transform(model_loaded.predict(
    np.expand_dims(test_set_normalized[-window_size:], axis=0)))
     

1/1 [==============================] - 1s 1s/step


In [96]:

for elem in tomorrow_pred[0]:
  print('{:.0f}'.format(elem))

3966


## setting batch size to 16 and see the result

## Adding a Callback for Early Stopping

also setting batch size to 16 and see the result

In [103]:
window_size = 20
batch_size = 16
shuffle_buffer_size = 1000

In [104]:
# Generate the dataset windows
windowed_training_ds_16 = windowed_dataset(training_set_normalized, window_size, 
                           batch_size, shuffle_buffer_size)
     

In [107]:
# Reset states generated by Keras
tf.keras.backend.clear_session()

model_tune2.set_weights(init_weights)

In [108]:
# set built parameter

print(training_set.shape)
model_tune2.build(input_shape = (1, window_size, training_set_normalized.shape[1]))
#model_tune2.summary()
# Set the training parameters
model_tune2.compile(loss=tf.keras.losses.Huber(), 
                    optimizer=tf.keras.optimizers.SGD(
                        learning_rate=learning_rate_value_all,
                        momentum=0.9))

(587,)


In [109]:
# Set the learning rate scheduler
lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: learning_rate_value_all * 10**(epoch / 20))

# Train the model
history_tune2 = model_tune2.fit(windowed_training_ds_16, epochs=epoch_value, 
                                callbacks=[lr_schedule],
                                verbose=0)

In [110]:
plot_loss_inlog_plotly(history_tune2, epoch_value, lrs_value=learning_rate_value_all)

### Train the Model

In [111]:
# Reset states generated by Keras
tf.keras.backend.clear_session()
model_tune2.set_weights(init_weights)
model_tune2.build(input_shape = (1, window_size, training_set_normalized.shape[1]))

# Set the training parameters
# usinbg mae as metrics
model_tune2.compile(loss=tf.keras.losses.Huber(), 
                    optimizer=tf.keras.optimizers.SGD(
                        learning_rate=0.0031,
                        momentum=0.9), metrics=["mae"])

In [112]:
# Train the model
history_build1 = model_tune2.fit(windowed_training_ds_16, epochs=epoch_value, 
                                verbose=0)

In [113]:
plot_series_plotly(np.arange(0, epoch_value), 
                   (history_build1.history['loss'], history_build1.history['mae']),
                   figure_title='Mae and loss value',
                   series_name=['Loss Value', 'MAE Value'],
                   xaxis_title='epoch', yaxis_title='Loss')

In [114]:
test_set_g2, results2 = plot_prediction_graph_plotly(
    model_tune2, df, training_ds_rows, 
    window_size, None, None,
    scaler=minmax_scaler,
    variable_names=['Close'],
    series_name=['test set (close)',
                 'pred set (close)']
    )

dateset_to_forecast shape: (167, 1)
expanded dateset_to_forecast shape: (1, 20, 1)
forecast shape (147, 1, 1)
forecast squeezed shape (147,)
test_set.size (147, 1)
results.ndim 1


In [115]:
print(history_build1.history['loss'][-1])
print(tf.keras.metrics.mean_squared_error(test_set_g2[:,-1], results2[:,-1]).numpy())
print(tf.keras.metrics.mean_absolute_error(test_set_g2[:,-1], results2[:,-1]).numpy())

0.005528532899916172
13900.37
89.74771


Changing batch size from 32 to 16 can increase model performace quite a bit 

## Adding a Callback for Early Stopping

we also need validation set

In [116]:
# convert to tensor
test_set = tf.convert_to_tensor(df_test['Close'])
test_set_normalised = minmax_scaler.transform(np.expand_dims(test_set, axis=1))

In [117]:
plot_series_plotly(df_test['Date'], test_set_normalised[:,0],
                   series_name='Close', figure_title='Data test Normalized')

In [121]:
print(test_set_normalized.shape)

(147, 1)


In [118]:
## Prepare features and labels (windowed data)

# Generate the dataset windows
windowed_test_ds = windowed_dataset(test_set_normalized, window_size, 
                           batch_size, shuffle_buffer_size)
     

In [123]:
# Print properties of a single batch
for windows in windowed_test_ds.take(1):
  print(f'data type: {type(windows)}')
  print(f'number of elements in the tuple: {len(windows)}')
  print(f'shape of first element: {windows[0].shape}')
  print(f'shape of second element: {windows[1].shape}')
  print(f'shape of first element expanded: {tf.expand_dims(windows[0], axis=-1).shape}')

data type: <class 'tuple'>
number of elements in the tuple: 2
shape of first element: (16, 20, 1)
shape of second element: (16, 1)
shape of first element expanded: (16, 20, 1, 1)


define a callback function that is run every end of an epoch. Inside, you will define the condition to stop training. For this lab, you will set it to stop when the val_mae is less than 5.2.

In [126]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    '''
    Halts the training when a certain metric is met

    Args:
      epoch (integer) - index of epoch (required but unused in the function definition below)
      logs (dict) - metric results from the training epoch
    '''

    # Check the validation set MAE
    if(logs.get('val_mae') < 0.01):

      # Stop if threshold is met
      print("\nRequired val MAE is met so cancelling training!")
      self.model.stop_training = True



In [127]:
# Instantiate the class
callbacks = myCallback()

Remember to set an appropriate learning rate here. If you're starting from random weights, you may want to use the same rate you used earlier. If you did not reset the weights however, you can use a lower learning rate so the model can learn better. If all goes well, the training will stop before the set 500 epochs are completed.

In [128]:
# Reset states generated by Keras
tf.keras.backend.clear_session()

model_tune2 = TS_LSTM_Model(window_size=window_size,
                            feature_size=training_set_normalized.shape[1])
#model_tune2.load_weights('./checkpoints/my_checkpoint')

model_tune2.set_weights(init_weights)
model_tune2.build(input_shape = (1, window_size, training_set_normalized.shape[1]))

# Set the training parameters
# usinbg mae as metrics
model_tune2.compile(loss=tf.keras.losses.Huber(), 
                    optimizer=tf.keras.optimizers.SGD(
                      learning_rate=0.003548,
                      momentum=0.9), 
                    metrics=["mae"])


set epoch to 500

In [129]:
# Train the model
history_build_with_callbacks = model_tune2.fit(windowed_training_ds,
                                 epochs=500,
                                 validation_data=windowed_test_ds,
                                 callbacks=[callbacks], 
                                 verbose=0)

In [130]:
history_build_with_callbacks.history.keys()

dict_keys(['loss', 'mae', 'val_loss', 'val_mae'])

In [134]:
plot_series_plotly(np.arange(0, epoch_value), 
                   (history_build_with_callbacks.history['loss'], history_build_with_callbacks.history['mae']),
                   figure_title='Mae and loss value',
                   series_name=['Loss Value', 'MAE Value'],
                   xaxis_title='epoch', yaxis_title='Loss')

In [135]:
plot_series_plotly(np.arange(0, epoch_value), 
                   (history_build_with_callbacks.history['val_loss'], 
                    history_build_with_callbacks.history['val_mae']),
                   figure_title='Mae and loss Validation value',
                   series_name=['Validation Loss Value', 'Validation MAE Value'],
                   xaxis_title='epoch', yaxis_title='Loss')

In [136]:
plot_series_plotly(np.arange(0, epoch_value), 
                   (history_build_with_callbacks.history['loss'], 
                    history_build_with_callbacks.history['val_loss']),
                   figure_title='Training Loss value and  Validation loss value',
                   series_name=['Training Loss Value', 'Validation Loss Value'],
                   xaxis_title='epoch', yaxis_title='Loss')

In [137]:
plot_series_plotly(np.arange(0, epoch_value), 
                   (history_build_with_callbacks.history['mae'], 
                    history_build_with_callbacks.history['val_mae']),
                   figure_title='Training MAE value and  Validation MAE value',
                   series_name=['Training MAE Value', 'Validation MAE Value'],
                   xaxis_title='epoch', yaxis_title='MAE Value')

In [138]:
test_set_g2, results2 = plot_prediction_graph_plotly(
    model_tune2, df, training_ds_rows, 
    window_size, None, None,
    scaler=minmax_scaler,
    variable_names=['Close'],
    series_name=['test set (close)',
                 'pred set (close)']
    )

dateset_to_forecast shape: (167, 1)
expanded dateset_to_forecast shape: (1, 20, 1)
forecast shape (147, 1, 1)
forecast squeezed shape (147,)
test_set.size (147, 1)
results.ndim 1


In [141]:
print(history_build_with_callbacks.history['loss'][-1])
print(history_build_with_callbacks.history['mae'][-1])
print(history_build_with_callbacks.history['val_loss'][-1])
print(history_build_with_callbacks.history['val_mae'][-1])
print(tf.keras.metrics.mean_squared_error(test_set_g2[:,-1], results2[:,-1]).numpy())
print(tf.keras.metrics.mean_absolute_error(test_set_g2[:,-1], results2[:,-1]).numpy())

0.004013397265225649
0.0685846209526062
0.003530886024236679
0.06337648630142212
8517.185
70.11361


In [142]:
# import time
# for x in range(4):
#   time.sleep(2*60*60)

## Tomorrow Pred

In [143]:
test_set = tf.convert_to_tensor(df_test[['Close']])

In [144]:
with np.printoptions(precision=0, suppress=True):
    print(test_set[-1].numpy())

[3880.]


In [145]:
test_set_normalized = minmax_scaler.transform(test_set)
test_set_normalized[-1]

array([0.19457014])

In [146]:
tomorro_pred = minmax_scaler.inverse_transform(model_tune2.predict(
    np.expand_dims(test_set_normalized[-window_size:], axis=0)))

with np.printoptions(precision=0, suppress=True):
    print(tomorro_pred)

1/1 [==============================] - 0s 55ms/step
[[3978.]]
